# 枚举,类型注释,dataclass数据类和模式匹配

## Enum枚举

枚举是3.4以后添加的，很容易使用，这里记录几个容易错误的点：

In [21]:
from enum import Enum


class Color(Enum):
    GREEN = 1
    YELLOW = 2
    RED = 3
    BLUE = 1

枚举的意义在于标签而不是数值，`print(color.GREEN)`打印的是`Color.GREEN`而不是1：

In [22]:
print(Color.GREEN)

Color.GREEN


### 枚举类型，名称和值

我们来梳理几个概念，`Color.GREEN`是枚举类型，每个枚举类型都有标签`name`和值`value`属性，`Color.GREEN.name`返回的是`str`，`Color.GREEN.value`返回的是`int`：

In [17]:
Color.GREEN

<Color.GREEN: 1>

In [18]:
Color.GREEN.name

'GREEN'

In [19]:
Color.GREEN.value

1

### 枚举的比较

枚举类型之间不能进行`>`或者`<`的比较，只能进行等值`==`或者身份`is`比较，比较的其实是枚举类型的值：

In [26]:
Color.GREEN == Color.BLUE

True

In [27]:
Color.GREEN is Color.BLUE

True

不同枚举类之间的枚举类型比较返回全部都是`False`：

In [29]:
class C(Enum):
    GREEN = 1

In [30]:
C.GREEN == Color.GREEN

False

### 枚举类型别名

如果枚举类型的值相同，那么后面的枚举类型是前面的别名：

In [33]:
from enum import Enum


class Color(Enum):
    GREEN = 1
    YELLOW = 2
    RED = 3
    BLUE = 1  # BLUE是GREEN的别名

遍历枚举类的时候，别名不会被遍历到：

In [36]:
for e in Color:
    print(e)

Color.GREEN
Color.YELLOW
Color.RED


枚举类的`__members__`属性包含了所有的枚举类型：

In [37]:
Color.__members__

mappingproxy({'GREEN': <Color.GREEN: 1>,
              'YELLOW': <Color.YELLOW: 2>,
              'RED': <Color.RED: 3>,
              'BLUE': <Color.GREEN: 1>})

### IntEnum和unique装饰器

普通`Enum`类，枚举类型的值可以是任意类型，`IntEnum`，枚举类型的值只能是整数。另外，不同的枚举类型的值可以是相同的，其中一个是另一个的别名，如果要保证值不同，可以使用`@unique`装饰器：

In [43]:
from enum import IntEnum, unique

@unique
class Color(IntEnum):
    GREEN = 1
    RED = 2
    YELLOW = 3
    # BLUE = 'blue' # IntEnum不允许值为非整数
    # BLUE = 1  # 添加了unique装饰器以后会报错

### 使用场景

#### 数值和枚举类型的转换

我们数据库里面为减少存储空间，一般都存储整数，比如用户性别，男性用0表示，女性用1表示，但是这样可读性不好，此时可以使用枚举类：

In [44]:
class SEX(IntEnum):
    MALE = 0
    FEMALE = 1

我们可以直接将数值转换为枚举类型，增加可读性：

In [52]:
male = SEX(0)
male

<SEX.MALE: 0>

## 模式匹配

3.10最重要的新增语法特性是`match/case`模式匹配，看起来有点像javascript的`switch/case`，但是功能要强大很多。

- [Python case|match - structural pattern matching](https://www.mybluelinux.com/python-casematch-structural-pattern-matching/)

### 匹配序列

匹配序列挺好理解的，不过python的模式匹配除了判断条件，还和解包有点类似，可以赋值给变量：
```python
metro_areas = [
    ('Tokyo', 'JP', 36.933, (35.689722, 139.691667)),
    ('Delhi NCR', 'IN', 21.935, (28.613889, 77.208889)),
    ('Mexico City', 'MX', 20.142, (19.433333, -99.133333)),
    ('New York-Newark', 'US', 20.104, (40.808611, -74.020386)),
    ('São Paulo', 'BR', 19.649, (-23.547778, -46.635833)),
]

for record in metro_areas:
    match record:
        case [city, _, _, (lat, lon)] if lon < 0:
            print(f"{city:15} | {lat:9.4f} | {lon:9.4f}")
```
我们甚至可以这样写，来指定要匹配的类型：
```python
case [str(city), *_, (float(lat), float(lon))] if lon < 0:
```
其中，`*_`表示匹配多个条目。注意一点，其实是先解构然后再执行`if`语句，因此在`if`里面可以使用变量。

### 匹配字典

模式匹配还可以匹配字典，比如：
```python
d = {0: "zero", 1: "one", 2: "two", 3: "three"}

match d:
    case {2: "two"}:
        print("yes")
```
此时表示，只要字典包含`{2: "two"}`的键值对，则认为匹配。

同样，可以进行变量赋值：
```python
d = {0: "zero", 1: "one", 2: "two", 3: "three"}

match d:
    case {0: zero_val, 1: one_val}:
        print(f"0 mapped to {zero_val} and 1 to {one_val}")
```
类似解包，还可以使用`**`号收集多余的项：
```python
d = {0: "zero", 1: "one", 2: "two", 3: "three"}
match d:
    case {2: "two", **remainder}:
        print(remainder)
```
输出为：`{0: 'zero', 1: 'one', 3: 'three'}`

注意，只能把变量绑定给值，不能绑定给键。下面这样是错的：
```python
d = {0: "zero", 1: "one", 2: "two", 3: "three"}
match d:
    case {two: "two"}:
        print(two)
```

### 匹配类实例

类模式匹配有三个变种，简单的、关键字和基于位置的。

#### 简单的类实例

```python
class C:
    pass


class SubC(C):
    pass


sc = SubC()
match sc:
    case C():
        print("match")
    case _:
        print("not match")
```
输出为`match`，说明模式匹配检查`sc`是不是`C`的实例，注意这里只能是`C()`，不能是`C`，如果是`C`的话，python会认为`C`是一个变量，同时将`sc`绑定给变量。

匹配类实例也可以绑定变量，前面已经见识过：
```python
case [str(city), *_, (float(lat), float(lon))] if lon < 0:
```
注意，上面这种绑定变量只适用于9种内置的类型：
```
bytes dict float frozenset int list set str tuple
```
如果不是这9种，则会认为是类的属性，就像他们是关键字或者位置参数，这句话不太好理解，继续往下看。

#### 关键字模式

类模式匹配的语法说实话有点奇怪，比如：
```python
from collections import namedtuple

Hero = namedtuple('Hero', 'name male')
widow = Hero('black widow', 'female')

match widow:
    case Hero(name='black widow'):
        print("match")
    case _:
        print("not match")
```
这里，只要类实例的`name`属性是`black widow`则认为匹配，不管有没有其它属性。其它属性还可以绑定变量：
```python
from collections import namedtuple

Hero = namedtuple('Hero', 'name sex')
widow = Hero('black widow', 'female')

match widow:
    case Hero(name='black widow', sex=sex):
        print(f"match, sex is: {sex}")
    case _:
        print("not match")
```
这里把`window`的`sex`属性绑定给了`sex`的变量。

#### 位置模式

位置模式和关键字参数基本一致，只不过参数按照位置传递：
```python
from collections import namedtuple

Hero = namedtuple('Hero', 'name sex')
widow = Hero('black widow', 'female')

match widow:
    case Hero('black widow', sex):
        print(f"match, sex is: {sex}")
    case _:
        print("not match")
```
要注意，模式匹配的位置参数和构造函数的参数不是一回事，它的位置在背后是由这个类的一个特殊的魔术方法`__match__args__`定义的，自定义的类是没有这个方法的，因此不能使用位置模式进行匹配：
```python
class C:
    def __init__(self, name, age):
        self.name = name
        self.age = age


c = C("spider man", 16)

match c:
    case C('spider man', age):
        print(f"match, age is: {age}")
    case _:
        print("not match")
```
上面代码会报错，提示`C() accepts 0 positional sub-patterns (2 given)`，因为自定义的类没有`__match_args__`方法，因此匹配的时候不接受位置参数。上面的代码，如果使用关键字参数是可以的：
```python
case C(name='spider man', age=age)
```
因此，在模式匹配中，不能把`()`号理解成构造函数。

## DataClass数据类

### 为什么需要数据类

通常情况下，我们使用命名元组或者字典来保存数据，但有时候我们希望像类一样，为数据提供一些方法，有点类似ORM映射。通常是这样做的：

In [1]:
class Coordinate:
    def __init__(self, lat, lon):
        self.lat = lat
        self.lon = lon

但是这显然有几个问题：
1. 字符串显示不友好
2. 实例默认彼此之间不相等

In [7]:
moscow = Coordinate(55.76, 37.62)
moscow  # 打印信息不友好

In [8]:
loc = Coordinate(55.76, 37.62)
moscow == loc  # 默认不相等

False

In [9]:
(loc.lat, loc.lon) == (moscow.lat, moscow.lon)  # 只能够捉对比较属性

True

我们看看使用命名元组和字典来实现这个数据结构：

In [12]:
# 使用命名元组
from collections import namedtuple

Coordinate = namedtuple('Coordinate', 'lat lon')
moscow = Coordinate(55.76, 37.62)
moscow  # 命名元组打印信息非常友好

Coordinate(lat=55.76, lon=37.62)

In [13]:
moscow == Coordinate(lat=55.76, lon=37.62)  # 相等也是有意义的

True

In [14]:
issubclass(Coordinate, tuple) # 命名元组是元组的子类

True

3.7以后还提供了一个类型命名元组，可以提供类型注释：

In [21]:
from typing import NamedTuple
Coordinate = NamedTuple('Coordinate', [('lat', float), ('lon', float)])
# 也可以使用关键字参数
# Coordinate = NamedTuple('Coordinate', lat=float, lon=float)
moscow = Coordinate(55.76, 37.62)
moscow

Coordinate(lat=55.76, lon=37.62)

其它特性和`namedtuple`一样，不过可以额外通过`typing.get_type_hints`方法获取其类型注释：

In [22]:
import typing
typing.get_type_hints(moscow)

{'lat': float, 'lon': float}

3.6以后还可以采用继承`Namedtuple`的方式写一个类：

In [25]:
# 注意：这里看起来是继承了NamedTuple，但是实际上是利用了元类，创建的并不是一个NamedTuple子类，而是一个tuple的子类
class Coordinate(NamedTuple):
    lat: float
    lon: float

In [28]:
moscow = Coordinate(55.76, 37.72)
moscow

Coordinate(lat=55.76, lon=37.72)

In [31]:
moscow == Coordinate(lat=55.76, lon=37.72)

True

最后再来看字典：

In [35]:
moscow = {"lat":55.76, "lon":37.72}
moscow

{'lat': 55.76, 'lon': 37.72}

In [36]:
moscow == {"lat":55.76, "lon":37.72}

True

可见命名元组和字典可以较好的完成任务。

3.7新增了一个`dataclass`类：

In [38]:
from dataclasses import dataclass

@dataclass
class Coordinate:
    lat: float
    lon: float

In [39]:
moscow = Coordinate(55.76, 37.62)
moscow

Coordinate(lat=55.76, lon=37.62)

In [40]:
moscow == Coordinate(55.76, 37.62)

True

In [41]:
typing.get_type_hints(moscow)

{'lat': float, 'lon': float}

### 数据类特性

dataclass除了和命名元组类似的作用以外，还提供了更多的功能：

#### 可变不可变

默认情况下，dataclass创建的数据类是可变的。比如：

In [45]:
moscow.area = 422000
# 此时添加的属性不会打印
moscow

Coordinate(lat=55.76, lon=37.62)

可以给dataclass装饰器添加一个frozen参数，使之称为不可变的：

In [56]:
from dataclasses import dataclass

@dataclass(frozen=True)
class Coordinate:
    lat: float
    lon: float

In [57]:
from dataclasses import FrozenInstanceError

moscow = Coordinate(lat=55.76, lon=37.62)
try:
    moscow.area = 422000
except FrozenInstanceError as e:
    print(e)

cannot assign to field 'area'


#### 获取字段

可以使用`dataclasses`的fields方法获取数据类的字段，返回一个`field`对象，包含这个字段的所有信息：比如字段名，默认值等等。

In [67]:
import dataclasses

[f.name for f in dataclasses.fields(moscow)]

['lat', 'lon']

如果要获取字段的类型，可以查看`__annotations__`属性，不过建议使用`typing.get_type_hints`方法：

In [68]:
moscow.__annotations__

{'lat': float, 'lon': float}

In [71]:
typing.get_type_hints(moscow)

{'lat': float, 'lon': float}

#### 修改字段值

当`frozen`设置为`True`的时候，数据类是不可变的，但是如果想要修改其中某个字段的值，我们可以使用`repalce`方法，该方法会返回一个新的数据类实例：

In [76]:
# 不能对字段进行赋值
try:
    moscow.lat = 58.75
except FrozenInstanceError as e:
    print(e)

cannot assign to field 'lat'


In [77]:
# 返回一个新的实例
dataclasses.replace(moscow, lat=58.75)

Coordinate(lat=58.75, lon=37.62)

#### 动态创建数据类

对于`namedtuple`和`NamedTuple`，可以使用构造函数动态的创建命名元组。`dataclasses`也提供`make_dataclass`方法，可以传入参数动态创建数据类：

In [79]:
Coordinate = dataclasses.make_dataclass('Coordinate', [('lat', float),
                                                       ('lon', float)])

In [81]:
moscow = Coordinate(lat=55.76, lon=37.62)
moscow

Coordinate(lat=55.76, lon=37.62)

#### 数据类细节

可以把数据类的类属性认为是实例属性的默认值，只要在类属性中添加了注释，则会在数据类实例中创建相应的实例属性：

In [1]:
from dataclasses import dataclass

@dataclass
class DemoDataClass:
    a: int          # 不会创建类属性a，但是会要求创建对应的实例属性b
    b: float = 1.1  # 会创建类属性b，也会要求创建对应的实例属性b
    c = 'spam'      # 普通的类属性，对实例属性没有约束

In [24]:
try:
    DemoDataClass.a  # 并不存在a这样的类属性
except AttributeError as e:
    print(e)

type object 'DemoDataClass' has no attribute 'a'


In [21]:
try:
    dc = DemoDataClass()
except TypeError as e:
    print(e)

__init__() missing 1 required positional argument: 'a'


In [22]:
dc = DemoDataClass(42)
dc.__dict__  # 创建了实例属性a,b

{'a': 42, 'b': 1.1}

#### dataclass装饰器签名

`dataclass`装饰器完整的签名是：
```python
@dataclass(*, init=True, repr=True, eq=True, order=False, unsafe_hash=False, frozen=False)
```
其中除了`frozen`意外，其它的参数表示是否自动生成对应的方法，比如`order`，表示是否自动生成`__lt__`，`__gt__`，`__le__`，`__ge__`方法。

In [48]:
# 如果init为False，则不会自动生成__init__方法
@dataclass
class C:
    a: int = 42
        
    def __init__(self):
        self.b = 33

c = C()
c.__dict__ 

{'b': 33}

`unsafe_hash`需要提一嘴，当设置了`frozen=True`，即为不可变时，会自动生成`__hash__`方法。但如果`frozen=False`，则`__hash__`为None。但是如果你仍然想自动生成`__hash__`方法，可以设置`unsafe_hash`为`True`，此时会根据所有`field`的值计算一个hash值，但是由于是可变的，所以要格外小心：

In [83]:
@dataclass(unsafe_hash=True)
class C:
    a: int = 42

In [84]:
c1 = C()
hash(c1)

-3075770106605038476

In [85]:
c2 = C()
hash(c2)

-3075770106605038476

In [86]:
c1 == c2

True

### Field字段

使用可变对象作为默认值往往是造成bug的源头，因此dataclass就不允许这种行为：

In [4]:
from dataclasses import dataclass

try:
    @dataclass
    class C:
        lst: list = []
except ValueError as e:
    print(e)

mutable default <class 'list'> for field lst is not allowed: use default_factory


错误信息很明确，使用`default_factory`：

In [6]:
from dataclasses import field

@dataclass
class C:
    lst: list = field(default_factory=list)

此时，在生成数据类实例的时候，会给`lst`传入一个空列表的实参，而不是使用空列表作为`lst`的默认参数：

In [9]:
c = C()
c.lst

[]

`field`可以接收很多参数，以下是一些场景，比如某个字段你不想打印出来，同时你想提供默认值，则可以：

In [11]:
@dataclass
class C:
    name: str
    leader: bool = field(default=False, repr=False)
        
c = C('shy')
c

C(name='shy')

In [12]:
c.leader

False

### `__post_init__`

有时候，我们希望对字段进行验证或者生成一些计算属性，则可以使用`__post_init__`方法，这个方法会在数据类自动生成的`__init__`的最后一步调用，比如：

In [14]:
@dataclass
class ClubMember:
    name: str
    guests: list = field(default_factory=list)
    athlete: bool = field(default=False, repr=False)
        
@dataclass
class HackerClubMember(ClubMember):
    all_handles = set()
    handle: str = ''
        
    def __post_init__(self):
        cls = self.__class__
        if self.handle == '':
            self.handle = self.name.split()[0]
        if self.handle in cls.all_handles:
            msg = f'handle {self.handle!r} already exists.'
            raise ValueError(msg)
        cls.all_handles.add(self.handle)

In [15]:
leo = HackerClubMember('Leo Rochael')
leo

HackerClubMember(name='Leo Rochael', guests=[], handle='Leo')

### 注释类属性

在数据类中，只要添加了类型注释，则会自动的作为参数传入`__init__`方法，生成实例属性。如果想创建一个类属性，则得像下面这样，借助`ClassVar`：

In [21]:
from typing import ClassVar

@dataclass
class C:
    members: ClassVar[list[str]] = []

此时不会创建`members`实例属性：

In [22]:
c = C()
c.__dict__

{}

### 非字段的初始化变量

有时候我们就需要给`__init__`传入一些参数，但是又不希望这个参数成为实例属性。此时可以使用`InitVar`，注意,`InitVar`是从`dataclasses`里导入的，`__init__`会将这些参数继续传递给`__post_init__`：
```python
from dataclasses import InitVar

@dataclass
class C:
    i: int
    j: int = None
    database: InitVar[DatabaseType] = None
    
    def __post_init__(self, database):
        if self.j is None and database is not None:
        self.j = database.lookup('j')
```

## 类型注释

因为python是动态语言，类型对其来说并不重要。而且对于python的鸭子类型来说，类型的定义和静态语言都有所不同，把python的某种类型理解成实现了某个特定方法的对象可能更合适。

### Subtype-of关系

传统的面对对象的静态语言系统，依赖Subtype-of关系。即如果T2是T1的子类，则T2是T1的子类型，比如：

In [4]:
class T1:
    pass

class T2(T1):
    pass

def func(o: T1):
    pass

func(T2)  # 因为T2 is subtype-of T1，所以检查通过，不会报错

如果定义时，参数规定是T1类型，那么调用的时候，可以传入T2作为参数，可以这么理解，因为定义时希望传入的参数是T1类型，而T2是T1的子类，可以实现所有T1的方法，因此T2显然是可以作为参数传递的。因此，我们把T2称为T1的子类型。

用通俗的话来讲，儿子总比老子强，任何子类型都可以代替父类型，这就是所谓的里氏替换原则。

### The Any type和Consistent-of关系

想象一下我们要注释以下的函数，假设`x`可以接受任意的类型：

In [1]:
def double(x: object)->object:
    return x * 2

使用mypy检查的话，会报错，因为mypy其实检查的是x是否支持`*`操作，或者说是否有`__mul__`方法。所以，如果是接受任意的类型，正确的做法是使用`Any`类型：

In [3]:
from typing import Any

def double(x: Any) -> Any:
    return x * 2

如果知道里氏替换原则，那么`Any`就显得非常违和，因为注释为`Any`的参数必须是任何类型的子类型，也就是说是任何类的子类。因此，python中，换了一个名词来定义类型和类型之间的关系，即兼容（consistent-of）关系，它适用于subtype-of，并对Any类型有特殊规定：
1. T1和它的子类型T2，则T2兼容(consistent-of)T1。
2. 任何类型都兼容Any：你可以将每种类型的对象传递给声明为Any类型的参数。
3. Any兼容任何类型：当需要某种类型的参数时，你总是可以传递Any类型的对象。

简单来说，Any就代表任意类型，不管是在声明的时候，还是在调用传参的时候，只要传递Any类型的对象，就不会报错。

### Optional和Union

当参数为可变对象的时候，我们经常会把默认值设置为`None`，此时注释应该为`Optional[type]`，表示参数为`type`类型或者为`None`。另外，`Optional`其实是`Union[type, None]`的一个快捷方式。

关于`Union`没有太多好说的，就是代表或的一个关系。不过有一个细节要注意，`Union`应该包含彼此之间不兼容的类型，比如：
`Union[str, int]`，如果彼此之间包含兼容关系，比如`int`和`float`，那么`Union[int, float]`的写法就多余了。直接写成`float`就够了，因为`int`兼容`float`，任何声明为`float`的地方都可以传入`int`。

### 通用集合类型

#### 列表、集合

所谓的通用集合类型，就是指内置的`list`,`set`等，还有一些内置类型，平时比较少用到，可以查阅流畅的python第三版第8章节。可以通过如下的形式写类型声明：
```python
container[item]
```
比如`list[str]`，表示对象是一个`list`，包含的元素全部为`str`类型。注意，虽然`list`可以包含任意类型的元素，但是通常情况下，我们是对`list`里面的元素进行操作，所以元素应包含相同方法，应该是同类型的。也就是说，在python中，一般`list`保存的是同质的元素，而`tuple`才是保存异质的元素。

如果`list`包含多种类型的值，那么可以使用`Union`,就像这样：`list[Union[str, int]]`。

#### 元组

元组的类型声明很简单，`tuple[str, float, str]`这样按位置依次指定元素的类型。如果全部都是相同的类型，则可以`tuple[str, ...]`这样声明。

如果是命名元组，`typing`模块提供了一个`NamedTuple`的数据类型，使用方法也很简单，如下：

In [5]:
from typing import NamedTuple

class Coordinate(NamedTuple):
    lat: float
    lon: float

然后就可以像下面这样进行声明：

In [7]:
def geohash(lat_lon: Coordinate) -> str:
    pass

注意，`Coordinate`兼容`tuple[float, float]`，但是反过来不行。因此，可以在函数定义的时候使用`tuple[float, float]`的声明，调用的时候传入`Coordinate`。

#### 字典

字典的注释很简单，如下：

In [58]:
d: dict[str, str] = {"name": "telecomshy"}

中括号里接收两个类型参数，分别代表键的类型和值的类型，但是你可能会发现一个问题，就是如果值的类型不同怎么办，一个方法是使用`Union`，但是这样丢失了键和值之间的关系。python3.8以后可以使用`TypedDict`，具体查看进阶的内容。

### 抽象基类

原则上，一个函数的参数类型都应该用collections.abc模块的抽象类进行声明，而返回值都应该用具体的类，这样可以让函数更加灵活。

#### 键值对类型的参数

如果是键值对类型的参数话，那么最好是使用`abc`下的`Mapping`或者`MutableMapping`。

因为`Mapping`除了`dict`，`defaultdict`，`ChainMap`，还可以兼容`UserDict`的子类。如果是`dict`的话，则只能兼容`defaultdict`，`ChainMap`，不能兼容`UserDict`的子类，因为`UserDict`不是`dict`的子类型，他们是兄弟关系，都是`MutableMapping`抽象类的子类。

`MutableMapping`是`Mapping`的子类，如果确定不会使用到类似`pop`，`update`这样更改字典的操作，那么使用`Mapping`，否则就使用`MutableMapping`。

#### 序列类型的参数

如果是序列类型的参数，那么使用`Sequence`或者`Iterable`这样的抽象类型比`list`,`tuple`这样的具体类要好。只要实现了`__iter__`方法的就属于可迭代对象，而`Sequence`则是实现了`__getitem__`这些方法的类。

这两者有类似的地方，不过如果只是对参数进行迭代的话，或者要接受生成器，那么就使用`iterable`，如果还有切片或者使用`len`求长度之类的操作的话，就使用`Sequence`。比如：

In [5]:
from collections.abc import Iterable, Sequence


def func(it: Sequence):
    print(','.join([str(i) for i in it]))


z = zip([(1, 2, 3), (4, 5, 6)])

func(z)

((1, 2, 3),),((4, 5, 6),)


`zip`对象没有`__getitem__`方法，但是有`__iterable__`方法，函数只是对传入的参数进行迭代，因此使用`__iterable__`更好。对比下面的例子，看什么时候使用`Sequence`，什么时候使用`Iterable`。

In [6]:
def func(it: Iterable):
    print(it[0])  # 这里会提示错误，因为Iterable没有__getitem__方法，这里需要使用Sequence

最后，某些情况下，需要使用`Iterator`类型，这部分内容后续在补充。

#### Numbers

Python的内部类numbers定义了数字类型的层次结构，依次是Number，Complex，Real，Rational，Integral。这个层次结构，在运行时的类型检查中工作的很好：

In [6]:
x = 32
isinstance(x, numbers.Number)

True

但是很遗憾，在静态类型检查中，numbers的这些类型完全不起作用了，这是因为python的静态类型本质上是检查对象是否包含某个特定的方法，而numbers中的类型是不包含方法的，因此对于数字，目前能使用的，只能是原生的complex, float, int。

### 类型别名

你可以直接把类型声明赋值给一个变量，作为声明的别名：

In [8]:
FromTo = tuple[str, int, str]

但是在3.10以后，对于别名也有一个专门的声明，以便typechecker工具能够更好的工作：
```python
from typing import TypeAlias

FromTo: TypeAlias = tuple[str, int, str]
```

### 类型变量TypeVar

我们有时候会遇到这样的场景，即返回的值的类型和参数的类型一致。比如函数的参数接受字符串或者整数，当是字符串的时候，返回的是字符串，此时可以使用泛型：

In [21]:
from typing import TypeVar

T = TypeVar('T')

def func(i: T) -> T:
    pass

此时`T`表示一个泛型，只有在第一次调用时才进行绑定。

如果我要对泛型进行一些约束，比如`T`只能够为数字或者字符串，则可以添加更多的参数：

In [22]:
from decimal import Decimal
from fractions import Fraction

T = TypeVar('T', float, Decimal, Fraction, str)

但是这样会发现，这个约束太宽泛了，此时我们可以使用`bound`参数，指定类型的上限，只要传入的是上限类型的子类型就可以。

In [23]:
T = TypeVar('T', bound=float)

我们仔细思考下面这个例子，理解如果使用泛型以及`bound`参数：

In [26]:
from collections import Counter
from collections.abc import Iterable, Hashable
from typing import TypeVar


def mode(data):
    pairs = Counter(data).most_common(1)
    if len(pairs) == 0:
        raise ValueError('no mode for empty data')
    return pairs[0][0]

首先，`data`必须要是一个可迭代序列，而且由于内部`Counter`将序列的值全部转换成字典的键，说明序列的值只要是`Hashable`即可，那么可能会这样写：

In [25]:
def mode(data: Iterable[Hashable]) -> Hashable:
    ...

这样写很明显会有一个问题就是，type checker只会检查返回对象是不是包含`__hash__`方法，不会做更多的事情了。如果这个返回值是数字或者字符串，我们后续要使用这个返回值，那么IDE不会提示任何关于字符串或者数字的方法。因此，最好的做法是像下面这样使用泛型：

In [27]:
from collections import Counter


HashableT = TypeVar('HashableT', bound=Hashable)

def mode(data: Iterable[HashableT]) -> HashableT:
    pairs = Counter(data).most_common(1)
    if len(pairs) == 0:
        raise ValueError('no mode for empty data')
    return pairs[0][0]

关于`TypeVar`非常重要的是，直觉上，`TypeVar`代表一种类型，比如：下面的代码，会给我们这样的感觉：
```python
def func(arg1: T, arg2: T) -> None:
    pass

func("a", 24)  # 这里会提示应为str，但是收到为int
```
所以，可能会理所当然的认为，当它用于表示容器内部的元素类型时，意味着容器内部元素的类型要保持一致，比如：
```python
def func(arg1: list[T]) -> None:
    pass

func(["a", 42])  # 可能认为这里会报错
```
但是实际上，上面的代码没有报错。第三方工具会根据列表内部的元素去推断T的类型，有两种方式，一种是判断T是和所有元素类型都兼容的类型，mypy采用这种方式，所以它会推断T的类型为object，另一种类似`Union`，认为T是所有元素类型的并集，pycharm似乎是采用这种方式。这回导致以下的结果：
```python
def func(arg: list[T]) -> T:
    return 42


a = func(["hello", 42])
b = a + 42
```
mypy会认为T为object,所以a也为object，a + 42会提示错误。而pycharm会认为T相当于`Union[str, int]`，所以不会报错。pycharm的这种方式更容易被理解。

另外，对于上面的代码，mypy还会抛出一个错误，会认为返回的42是`int`和`T`不兼容。mypy会检查`return`返回的结果，只有返回`arg[0]`这样的形式才不会报错。甚至如下的代码：
```python
def func(arg: T) -> T:
    return 42
```
mypy也报错，只有原样返回`arg`，mypy才不会报错。但是pycharm不会。

但是pycharm也有缺陷，它似乎并不检查函数实际返回的值的类型，而是仅仅以声明的返回类型为准：
```python
def func(arg: T) -> T:
    return 42


a = func("hello")
```
这样的情况，pycharm是不报错的，再输入`b = a + 42`，pycharm会提示错误，因为它仅根据声明判断返回的值`a`为字符串，会提示`a + 42`有错误。

### AnyStr

AnyStr是一个语法糖，相当于：

In [28]:
from typing import TypeVar

AnyStr = TypeVar('AnyStr', str, bytes)

### 静态协议

我们先来考虑下面的一段代码，想一下我们应该如果给它添加类型声明：

In [29]:
def top(series, length):
    ordered = sorted(series, reverse=True)
    return ordered[:length]

很明显，series是序列，length应该是个整型：

In [31]:
from collections.abc import Iterable


def top(series: Iterable, length: int) -> list:
    ordered = sorted(series, reverse=True)
    return ordered[:length]

这样是可以的，但是太宽泛，series里可以是任意的值，这有两个缺点：
1. 如果series里的值输入错误，type checker并不能发现错误。
2. 如果我们后续要使用返回的值，由于可以是任意值，IDE不会给我们什么提示。

可是如何对series里的元素进行注释呢？我们观察发现，函数主要是对series进行排序，我们可以传入一个`object`构成的序列：

In [32]:
sorted([object for _ in range(4)])

TypeError: '<' not supported between instances of 'type' and 'type'

错误提示很明确，序列里的对象需要支持`<`操作。现在我们可以编写一个静态协议：

In [33]:
from typing import Protocol, Any, TypeVar

class SupportsLessThan(Protocol):
    def __lt__(self, other: Any) -> bool: ...

In [34]:
LT = TypeVar('LT', bound=SupportsLessThan)

def top(series: Iterable[LT], length: int) -> list[LT]:
    ordered = sorted(series, reverse=True)
    return ordered[:length]

### Callable

如果参数是函数或者可调用的类型，那么可以使用`typing`的`Callable`类型：
```python
Callable[[ParamType1, ParamType2], ReturnType]
```
`Callable`需要注意的一点是，如果参数是动态的，可以使用`...`来标注，即：
```python
Callable[..., ReturnType]
```

### NoReturn

这是个特殊的类型，专门用来标注那些永远也不会返回值的函数。比如函数会抛出异常。

### 位置参数和可变参数

3.8新增了仅位置参数，类似以前`*`后面的必为关键字参数一样，在`/`前面的参数，在函数调用时，只能根据位置传参。我们看一个函数的签名：

In [38]:
from typing import Optional


def tag(
    name: str,
    /,
    *content: str,
    class_: Optional[str] = None,
    **attrs: str,
) -> str:
    pass

`*content: str`表示除`name`外的所有的其它位置参数都要是`str`类型。因此，在函数体内，`content`这个变量的类型声明为`tuple[str, ...]`。

`**attrs: str`表示所有关键字参数的值是`str`类型，在函数体内，`attrs`变量的类型声明为`dict[str, str]`，如果是`**attrs: float`，那么，在函数体内，`attrs`变量的类型声明为`dict[str, float]`。如果`attrs`要接收不同类型的值，那么可以使用`Union`或者`Any`，`**attrs: Any`。

### 静态声明的缺陷

静态声明可以帮助我们提前发现错误，可以帮助IDE给出代码提示，但是它仍然有缺陷，某些情况下它会出错。而且以下几种情况，静态声明表示无能为力：
- 参数解包，比如当我们使用`config(**settings)`调用函数时，静态检查是无法生效的。
- 一些高级的特性比如特性、描述符、元类，在元编程领域，静态检查能做的很有限。
- 类型声明无法发现一些数据约束，比如整数大于0，或者一个字符串，长度在6到12个字符之间这样的约束。

### 变量注释

#### 基本语法

除了函数的参数和返回值，某些情况下我们需要直接对变量进行类型声明，如下，两种写法都是可以的：

In [62]:
# 第一种写法
my_var: int
my_var = 5

In [63]:
# 第二种写法
other_var: int = 5

虽然两种写法效果都可以，但是如果有分支的情况下，第一种写法更好：

In [64]:
my_var: bool

if 2 + 2 == 4:
    my_var = True
else:
    my_var = False

注意，当元组打包的时候，语法允许变量提示，但是解包时，是不允许的：

In [65]:
from typing import Tuple

t: Tuple[int, ...] = (1, 2, 3)  # 省略号表示其它所有元素都是int类型
t: Tuple[int, ...] = 1, 2, 3  # 3.8版本以后也可以这样写

但是解包是不能直接写类型提示的：

In [66]:
a:int, b:int, c:int = (1, 2, 3)  # 不能这样写，提示语法错误

SyntaxError: invalid syntax (Temp/ipykernel_17180/434319214.py, line 1)

只能够每个变量先注释：

In [70]:
from typing import Optional

header: str
kind: int
body: Optional[list[str]]  # 表示body为str构成的列表或者为None
header, kind, body = ["header", 42, None]

#### 全局和局部的变量注释

注意，当全局变量注释时，会产生一个未绑定的变量：
```python
a: int
print(a)  # 抛出NameError，变量未定义
```
如果是在函数内被定义，那么会产生一个局部的未绑定的变量：
```python
a = 42

def func():
    a: int
    print(a) 

func()  # UnboundLocalError: local variable 'a' referenced before assignment
```

#### 类和实例的变量注释

类型注释还可以用于注释类主体和方法中的类和实例变量。特别是，无值表示法`a: int`允许对应该在`__init__`或`__new__`中初始化的实例变量进行注释。建议的语法如下所示:
```python
class BasicStarship:
    captain: str = 'Picard'               # instance variable with default
    damage: int                       # instance variable without default
    stats: ClassVar[Dict[str, int]] = {}      # class variable
```
虽然`caption`是类属性，但是其目的是所有实例变量设置一个默认值，所以在`__init__`中还需要定义一个`captain`的实例变量。但是，如果像下面这样写：
```python
class BasicStarship:
    captain: str = 42  # captain只会检查实例属性是否是str，不会检查类属性
```
此时是不会提示错误的，说实话，这挺容易让人产生误解的。而真正的类变量不应该被实例变量覆盖，因此需要使用`ClassVar`注释，此时如果意外设置了实例的`stats`属性，编辑器会提示错误。

也可以直接对实例变量进行注释：
```python
class BasicStarship:
    def __init__(self, captain):
        self.captain: str = captain
```
注意一点，ClassVar不能包含任何类型变量

### 类型注释进阶

类型注释的内容很多，因此分为两个部分，这部分的内容属于进阶内容。

#### 签名重载

2023/10/11更新：该章节解释不清楚，参考下面的文章：
- [Python Type Hints - How to Use @overload](https://adamj.eu/tech/2021/05/29/python-type-hints-how-to-use-overload/)
- [function overloading](https://mypy.readthedocs.io/en/stable/more_types.html#function-overloading)

有时候，我们会根据参数的类型返回不同的记过，此时可以使用`overload`装饰器对函数签名进行重载。比如内置的`sum`函数：

In [42]:
from typing import overload

@overload
def sum(__iterable: Iterable[_T]) -> Union[_T, int]: ...
@overload
def sum(__iterable: Iterable[_T], start: _S) -> Union[_T, _S]: ...

先看第一种，`start`采用默认值为0的情况，则返回值为参数类型或者为整型（当`__iterable`为空列表时），如果输入了`start`参数，则为第二种情况。

你可能会疑惑为什么不直接这样写：

In [44]:
def get_sum(it: Iterable[T], /, start: int = 0) -> Union[T, int]: ...

这样写的话，当给start赋值浮点数的时候，比如`get_sum([1, 2, 3], 2.3)`，mypy会报错，提示`Argument 2 to "get_sum" has incompatible type "float"; expected "int"`。而如果这样写：

In [45]:
def get_sum(it: Iterable[T], /, start: S = 0) -> Union[T, S]: ...

pycharm倒是不报错了，但是mypy会提示错误：`Incompatible default for argument "start" (default has type "int", argument has type "S")`。

如果参数有默认值，且调用时的值和默认值类型不一致的话，可能就需要使用使用`overload`定义函数。

所以，代码越灵活，注释声明就越难写。没有必要追求完美注释，python本来就是动态语言，所以要有取舍。

#### TypedDict

前面我们提到过，对普通的字典进行类型声明，存在缺陷。所以python3.8以后，typing模块提供了一个`TypedDict`的类型：

In [60]:
from typing import TypedDict

class Book(TypedDict):
    name: str
    auther: list[str]

但是，`TypedDict`的作用没有想象的那么大，它不能进行运行时的验证，仅仅只是为键值提供了对应的类型声明。特别是在对json,xml的转换中，流畅的python第二版15章详细讨论了`TypedDict`的缺点，这种场景下，最好使用`pydantic`库。

#### Type Cast

类型系统不是完美的，很多时候都只能根据代码进行推断，有些情况下会推断错误，这种情况下，可以使用`cast`函数人为指定类型，类型系统会相信你，从而不会报错。

In [61]:
def find_first_str(a: list[object]) -> str:
    index = next(i for i, x in enumerate(a) if isinstance(x, str))
    return cast(str, a[index])

这里，类型系统会认为`a[index]`是`object`类型，和返回的`str`不一致，会提示错误，因此需要手动的指定为`str`类型。

如果要屏蔽类型检查系统，还有几种方法：
1. 在注释中标明：`# type: ignore`
2. 使用`Any`类型

但是两种方法都有缺点，第一种方法不够明确，而第二个如果滥用的话可能会造成连锁效应，降低类型检测的作用。这三种方法各有应用场景，应根据实际情况进行选择。

#### 实现一个泛型类

这部分的内容目前比较少用到，后期再补充

#### 不变、逆变和协变

这部分的内容目前比较少用到，后期再补充

## 模式匹配

### 匹配序列

### 匹配字典

### 匹配实例